In [ ]:
!pip -q install kaggle scikit-learn pandas numpy matplotlib gradio transformers datasets accelerate
# 在Colab安裝此次專案會用到的所有套件（-q是少顯示訊息）

In [ ]:
import os, re, html, random
# 基本工具模組，用來處理檔案、文字與亂數
import numpy as np
# numpy：數值運算用，搭配 random seed 使用
import pandas as pd
# pandas讀取csv、整理資料成DataFrame
import matplotlib.pyplot as plt
# 繪製模型評估圖表
from sklearn.model_selection import train_test_split
# 把資料切成訓練集跟驗證集，後面所有模型都用同一份切法
from sklearn.metrics import f1_score
# 使用F1-score當評估指標
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# 將文字轉成模型可用的數值特徵
from sklearn.linear_model import LogisticRegression
# 傳統文字分類常用的baseline模型之一
from sklearn.naive_bayes import MultinomialNB
# 適合文字資料的Naive Bayes，常見的baseline
import torch
# PyTorch，後面BERT/RoBERTa模型實際運算用
from datasets import Dataset
# HuggingFace的Dataset格式，讓資料可以直接接Trainer
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
# AutoTokenizer自動載入對應模型的tokenizer
# AutoModelForSequenceClassification直接用預訓練模型做分類
# TrainingArguments設定訓練參數（learning rate、batch size等）
# Trainer：HuggingFace提供的高階訓練介面，不用自己寫training loop

# 這段主要是在準備會用到的工具，前面是傳統NLP baseline會用到的套件，後面是HuggingFace跑DistilBERT跟RoBERTa用的
# 將baseline與transformer的工具放在前面，是為了讓整個實驗流程比較清楚

In [ ]:
# 設定亂數種子
SEED = 42
# 固定亂數結果，確保每次實驗可重現
random.seed(SEED)
# 設定Python內建亂數來源
np.random.seed(SEED)
# 設定numpy的亂數來源

# 因為會做多個模型比較，如果每次亂數不固定，資料切分可能會不一樣，F1分數就會亂跳，這樣比較不公平
# 所以先把random跟numpy的seed都固定，確保每個模型是在同樣條件下比較

In [ ]:
# 建立Kaggle API金鑰的位置
!mkdir -p ~/.kaggle
# 把上傳的kaggle.json放到正確目錄
!cp kaggle.json ~/.kaggle/
# 設定讀取權限
!chmod 600 ~/.kaggle/kaggle.json
# 測試是否能正確執行Kaggle API
!kaggle --version

# 因為是用Colab跑實驗，所以需要透過Kaggle API抓比賽資料，所以先把kaggle.json放到Kaggle指定的路徑，再設定權限避免被拒絕
# 最後用kaggle --version確認API有成功連上

Kaggle API 1.7.4.5


In [ ]:
!kaggle competitions download -c nlp-getting-started
# 從Kaggle下載競賽的資料集
!unzip -o nlp-getting-started.zip
# 解壓縮下載的資料檔（-o表示直接覆蓋舊檔）

# 這部分是把Kaggle的原始資料抓下來，後面所有模型都是用同一份資料，確保公平

  0% 0.00/593k [00:00<?, ?B/s]
100% 593k/593k [00:00<00:00, 842MB/s]
Archive:  nlp-getting-started.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
train = pd.read_csv("train.csv")
# 讀取訓練資料集
test = pd.read_csv("test.csv")
# 讀取測試資料集
train.shape, test.shape
# 查看訓練集與測試集的資料筆數與欄位數
train.head()
# 快速檢視訓練資料前幾筆內容確認格式

# 在這裡先看shape，避免資料沒載好或欄位數不對，並且印出內容確認格式

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train[train['target']==0].sample(2)[['text','target']]
# 從標籤為0的資料中隨機抽兩筆來看文字內容與標籤

# 簡報需參考的資料

,text,target
3697,Everyday is a near death fatality for me on th...,0
4180,#Lifestyle Û÷It makes me sickÛª: Baby clothe...,0


In [ ]:
train[train['target']==1].sample(2)[['text','target']]
# 從標籤為1的資料中隨機抽兩筆來看災難相關的文字內容

# 簡報需參考的資料

,text,target
3162,Came across this fire video not mine..enjoy..#...,1
4558,#BreakingNews Militants attack Udhampur police...,1


In [ ]:
train['target'].value_counts()
# 查看各類別的樣本數量，確認資料是否有不平衡問題

# 簡報需參考的資料

,count
target,
0,4342
1,3271


In [ ]:
def clean_text_fixed(s: str) -> str:
# 定義一個固定的文字清洗函式，確保所有模型用到的前處理都一模一樣

    s = html.unescape(str(s))
    # 把文字中可能的HTML編碼轉回正常字元，避免奇怪符號影響模型
    s = s.lower()
    # 全部轉成小寫，避免同一個字因大小寫被當成不同特徵
    s = re.sub(r"http\S+|www\S+", " URL ", s)
    # 把網址統一換成URL標記，避免每個不同網址被當成不同詞
    s = re.sub(r"@[\w_]+", " USER ", s)
    # 把使用者標記（@某人）統一處理，避免人名干擾模型判斷
    s = re.sub(r"#[\w_]+", " HASHTAG ", s)
    # 把hashtag統一成HASHTAG，保留語意但不保留具體字詞
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    # 移除英文字母和數字以外的符號，讓輸入更乾淨
    s = re.sub(r"\s+", " ", s).strip()
    # 把多餘的空白合併，並去掉前後空白，避免影響向量化

    return s
    # 回傳清洗後的文字

# 這樣清理是為了讓模型專注在文字本身的語意，而不是被網址、人名或符號干擾
# 這個版本主要提供給傳統 baseline 與 DistilBERT 使用，確保所有模型比較時的公平性


from tqdm import tqdm
# 用來顯示文字清洗的進度條，避免長時間運行時不知道卡在哪

tqdm.pandas(desc="Cleaning text")
# 啟用 pandas 的 progress_apply，在處理大量文字時比較有感覺


def clean_teacher(s: str) -> str:
# 參考老師提供的文字清理概念，做較溫和但語意保留較完整的清洗版本
# 這個版本主要用於 RoBERTa，目的是提升語意模型的理解能力與Kaggle分數

    s = str(s)
    # 確保輸入一定是字串，避免出現NaN或非文字型態導致錯誤

    s = s.lower()
    # 將所有文字轉為小寫，避免同一單字因大小寫被視為不同token

    s = re.sub(r"http\S+|www\S+", "", s)
    # 移除網址本身，但不另外加標記，避免對語意模型造成干擾

    s = re.sub(r"@\w+", "", s)
    # 移除@使用者名稱，避免人名資訊影響模型判斷

    s = re.sub(r"#", "", s)
    # 保留hashtag後面的文字內容，只移除#符號，讓語意仍然存在

    s = re.sub(r"\s+", " ", s).strip()
    # 合併多餘空白並移除前後空白，讓輸入文字更乾淨

    return s
    # 回傳清洗後的文字

# 這個清洗方式不會過度破壞原始句子結構，特別適合Transformer類模型
# 與前面的 clean_text_fixed 並存，是為了讓不同模型使用最適合自己的前處理方式

In [ ]:
train["text_fixed"] = train["text"].apply(clean_text_fixed)
# 將訓練資料的原始文字套用剛剛定義的清洗函式，並存成新的欄位保留原始資料
test["text_fixed"]  = test["text"].apply(clean_text_fixed)
# 對測試資料做一模一樣的文字清洗，確保訓練與預測的輸入格式一致

In [ ]:
train["keyword_fixed"] = train["keyword"].fillna("").str.replace("%20", " ")
# 把keyword的缺值補成none，並把網址編碼的%20還原成正常空白，方便模型讀取
test["keyword_fixed"]  = test["keyword"].fillna("").str.replace("%20", " ")
# 對測試資料做完全一樣的keyword清理，避免訓練和預測格式不一致
train["input"] = train["keyword_fixed"] + " " + train["text_fixed"]
# 把keyword跟清洗後的推文文字接在一起，當作模型實際看到的輸入
test["input"]  = test["keyword_fixed"] + " " + test["text_fixed"]
# 測試資料也用相同的合併方式，確保模型預測時條件一致

# keyword是人工標註的重點詞，把它當成額外提示接到文字前面，看看能不能幫模型更快抓到災難相關訊息
# 不分開欄位是因為這樣合併成單一文字，對傳統模型跟BERT都比較好處理，也比較容易公平比較

In [ ]:
train["text_rb"] = train["text"].progress_apply(clean_teacher)
# 使用較溫和的文字清洗方式，保留較完整語意，RoBERTa使用

test["text_rb"]  = test["text"].progress_apply(clean_teacher)
# 測試資料也用相同清洗方式，確保訓練與預測條件一致

train["input_rb"] = train["keyword_fixed"] + " " + train["text_rb"]
# 將keyword與RoBERTa專用清洗後的文字合併，作為RoBERTa的輸入

test["input_rb"]  = test["keyword_fixed"] + " " + test["text_rb"]
# 測試資料同樣合併，確保推論時格式一致

Cleaning text: 100%|██████████| 3263/3263 [00:00<00:00, 14528.83it/s]


In [ ]:
X = train["input"].values
# 取出前面整理好的文字輸入（keyword + text），當作模型的X

y = train["target"].values
# 取出對應的標籤（0/1），當作模型要學的答案y

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    # 把資料分成訓練集和驗證集，後面模型都用這一組切法

    test_size=0.2,
    # 留20%當validation，用來評估模型表現，不參與訓練

    random_state=SEED,
    # 固定亂數種子，確保每次切出來的資料都一樣，結果可重現

    stratify=y
    # 依照標籤比例切資料，避免train或val某一類特別少
)

print("Train:", len(X_train), "Val:", len(X_val))
# 印出訓練集和驗證集的筆數，確認切分比例是否正確

# 切validation是因為要在同一個資料切分下公平比較不同模型，validation F1是用來選模型的。
# stratify=y是因為這是二分類問題，如果不分層可能會讓validation某一類太少，F1會不準。


Train: 6090 Val: 1523


In [ ]:
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
# 建立TF-IDF向量器，最多用5000個特徵，包含unigram跟bigram
X_train_vec = tfidf.fit_transform(X_train)
# 用訓練資料學習詞彙與權重，並把文字轉成數值向量
X_val_vec = tfidf.transform(X_val)
# 用同一組TF-IDF設定，把validation文字轉成向量（不能再fit）
logreg = LogisticRegression(max_iter=1000)
# 建立邏輯斯回歸模型，並把迭代次數拉高避免不收斂
logreg.fit(X_train_vec, y_train)
# 用訓練集的向量與標籤來訓練baseline模型
val_pred_lr = logreg.predict(X_val_vec)
# 用訓練好的模型預測validation資料的分類結果
f1_lr = f1_score(y_val, val_pred_lr)
# 計算validation的F1-score，當作baseline的評估指標
print("TF-IDF + LR | F1:", round(f1_lr, 4))
# 印出baseline模型的F1分數，方便後面跟其他模型比較

# 先用TF-IDF + Logistic Regression是因為這是NLP中很常見的baseline，先確認資料本身是不是有可學習的訊息
# 用bigram則是因為災難推文常有關鍵片語，bigram可以捕捉簡單的上下文資訊
# validation只用transform是因為不能讓validation的資訊影響訓練，避免data leakage

TF-IDF + LR | F1: 0.7682


In [ ]:
cv = CountVectorizer(max_features=5000, ngram_range=(1,2))
# 建立CountVectorizer，只記錄詞出現的次數，作為另一種baseline表示法
X_train_cv = cv.fit_transform(X_train)
# 用訓練資料建立詞彙表，並把文字轉成詞頻向量
X_val_cv   = cv.transform(X_val)
# 使用同一個詞彙表，把validation文字轉成詞頻向量

nb = MultinomialNB()
# 建立Multinomial Naive Bayes，是很常搭配文字詞頻的分類模型

nb.fit(X_train_cv, y_train)
# 用訓練集的詞頻向量與標籤訓練Naive Bayes模型

val_pred_nb = nb.predict(X_val_cv)
# 使用訓練好的模型預測validation資料的分類結果

f1_nb = f1_score(y_val, val_pred_nb)
# 計算Naive Bayes在validation上的F1-score

print("CountVec + NB | F1:", round(f1_nb, 4))
# 印出第二個baseline的F1分數，用來跟其他模型做比較


# CountVectorizer+NB是因為這是NLP很經典、計算速度快的baseline，想看看只用詞頻、不用權重時模型的表現
# 與TF-IDF+LR的差別是TF-IDF有考慮詞的重要性，而CountVectorizer只看出現次數，兩者代表不同特徵設計思路
# Naive Bayes適合文字是因為文字資料通常是非負整數的詞頻，Multinomial Naive Bayes假設剛好符合這種分布

CountVec + NB | F1: 0.7591


In [ ]:
model_name = "distilbert-base-uncased"
# 指定要使用的預訓練模型名稱，選的是較輕量的DistilBERT

tokenizer = AutoTokenizer.from_pretrained(model_name)
# 載入和DistilBERT對應的tokenizer，負責把文字轉成模型看得懂的token


def tokenize_fn(batch):
    # 定義一個函式，之後會拿來批次處理資料集的文字

    return tokenizer(
        batch["text"],
        # 指定要處理的欄位，這裡是我們的輸入文字

        padding="max_length",
        # 把所有句子補齊到固定長度，方便batch訓練

        truncation=True,
        # 如果句子太長就直接截斷，避免超過模型限制

        max_length=128
        # 設定每筆輸入最多128個token，在效能與速度間取得平衡
    )


# 使用tokenizer是因為BERT不能直接吃文字，一定要先轉成token和對應的編碼
# 使用DistilBERT是因為它是BERT的精簡版，速度快、資源需求低，但效果還不錯，適合用來比較
# padding是讓每筆資料長度一致，truncation是避免句子太長造成錯誤

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
ds_train = Dataset.from_pandas(pd.DataFrame({
    "text": X_train,
    "label": y_train
}))
# 把訓練資料的文字與標籤整理成DataFrame，再轉成HuggingFace的Dataset格式

ds_val = Dataset.from_pandas(pd.DataFrame({
    "text": X_val,
    "label": y_val
}))
# 同樣方式把validation資料轉成Dataset，方便之後用Trainer做評估

ds_train = ds_train.map(tokenize_fn, batched=True)
# 對訓練資料套用tokenizer，把文字轉成BERT需要的input_ids和attention_mask

ds_val   = ds_val.map(tokenize_fn, batched=True)
# 對validation資料做一樣的tokenization，確保前處理流程一致

cols = ["input_ids", "attention_mask", "label"]
# 指定模型訓練時真正會用到的欄位，避免保留多餘資料

ds_train.set_format("torch", columns=cols)
# 把訓練資料轉成PyTorch tensor，讓Trainer可以直接丟進模型訓練

ds_val.set_format("torch", columns=cols)
# 把validation資料也轉成PyTorch tensor，用於模型評估

# 這邊是把原本切好的文字資料轉成HuggingFace Trainer能用的格式，先tokenize成input_ids，再指定欄位轉成PyTorch tensor，後面就不用自己寫training loop


Map:   0%|          | 0/6090 [00:00<?, ? examples/s]

Map:   0%|          | 0/1523 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)
# 載入預訓練的DistilBERT，並設定成二元分類（0/1）的分類模型

def compute_metrics(eval_pred):
    # 定義一個評估函式，讓Trainer在驗證階段知道要如何算成績

    logits, labels = eval_pred
    # 從Trainer傳進來的結果中，取出模型輸出的logits跟真實標籤

    preds = logits.argmax(axis=1)
    # 對每筆資料選機率最大的那一類，當作模型的預測結果

    return {"f1": f1_score(labels, preds)}
    # 用預測結果跟真實標籤計算F1-score，回傳給Trainer顯示

# 這邊是設定分類模型本身，然後另外寫一個F1的計算方式，因為這個比賽跟作業都用F1當主要指標，所以就沒有用accuracy

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
args = TrainingArguments(
    output_dir="./distilbert_out",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=2,
    seed=SEED,
    report_to="none"
)
# 設定DistilBERT訓練時用到的所有超參數與基本訓練設定

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_train,
    eval_dataset=ds_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
# 使用HuggingFace的Trainer，把模型、資料、訓練設定跟評估方式全部包在一起

trainer.train()
# 正式開始對DistilBERT進行fine-tuning訓練
eval_distil = trainer.evaluate()
# 在validation dataset上評估訓練完成的DistilBERT模型表現
f1_distil = eval_distil["eval_f1"]
# 從評估結果中取出validation的F1-score
print("DistilBERT | F1:", round(f1_distil, 4))
# 印出DistilBERT在validation set上的F1分數，方便和其他模型比較

# 這部分是用HuggingFace的Trainer設定訓練流程，學習率跟epoch都是用常見的BERT fine-tuning設定，
# 最後用validation F1跟前面的baseline模型做比較

/tmp/ipython-input-12890169.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


DistilBERT | F1: 0.8068


In [ ]:
roberta_name = "roberta-base"
# 指定要使用的預訓練模型名稱
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_name)
# 載入RoBERTa對應的tokenizer，負責把文字轉成模型看得懂的token

# 因為RoBERTa跟BERT用的tokenizer不一樣，所以另外載入RoBERTa專用的tokenizer，確保文字切token的方式跟模型一致

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def roberta_tokenize(batch):
# 定義一個專門給RoBERTa用的文字轉換函式，之後Dataset會用到
    return roberta_tokenizer(
    # 呼叫剛剛載入的RoBERTa tokenizer，把文字轉成模型需要的格式
        batch["text"],
        # 從資料集中取出文字欄位，這裡的batch是一次處理多筆資料
        padding="max_length",
        # 把每筆文字補齊到固定長度，避免batch裡長度不一致造成錯誤
        truncation=True,
        # 如果文字超過最大長度，就直接截斷，避免超出模型限制
        max_length=160
        # RoBERTa對較長文本較穩定，稍微拉長觀察效果
    )
    # 回傳tokenized後的結果給HuggingFace Dataset使用

# 這個function是在做RoBERTa的tokenizer，長度設定調整為160，拉長觀察效果

In [ ]:
X_rb = train["input_rb"].values
# 取出 RoBERTa 專用的文字輸入（keyword + teacher 清洗後的 text）

X_rb_train, X_rb_val, _, _ = train_test_split(
    X_rb, y,
    test_size=0.2,
    random_state=SEED,
    stratify=y
)
# 與前面模型使用相同的切分比例，確保不同模型比較時的公平性

ds_train_rb = Dataset.from_pandas(pd.DataFrame({
# 把訓練資料整理成DataFrame，再轉成HuggingFace Dataset方便後面用Trainer

    "text": X_rb_train,
    # 放入前面切好的 RoBERTa 訓練文字資料（teacher cleaning + keyword）

    "label": y_train
    # 放入對應的訓練標籤，0/1分別代表非災難與災難
}))
# 建立完成RoBERTa用的訓練Dataset

ds_val_rb = Dataset.from_pandas(pd.DataFrame({
# 把validation資料用同樣方式轉成HuggingFace Dataset

    "text": X_rb_val,
    # 放入 RoBERTa validation 的文字資料，用來評估模型

    "label": y_val
    # 放入validation對應的真實標籤
}))
# 建立完成RoBERTa用的validation Dataset


ds_train_rb = ds_train_rb.map(roberta_tokenize, batched=True)
# 對訓練資料套用RoBERTa tokenizer，把文字轉成input_ids與attention_mask

ds_val_rb   = ds_val_rb.map(roberta_tokenize, batched=True)
# 對validation資料做一樣的tokenizer，確保前處理流程一致


ds_train_rb.set_format("torch", columns=cols)
# 指定訓練資料轉成PyTorch tensor，只保留模型需要的欄位

ds_val_rb.set_format("torch", columns=cols)
# validation資料也轉成PyTorch tensor，Trainer才能直接使用

# 這裡是把切好的訓練跟驗證資料轉成HuggingFace Dataset，
# 接著用RoBERTa tokenizer做前處理，
# 最後轉成PyTorch tensor，讓Trainer可以直接訓練模型

Map:   0%|          | 0/6090 [00:00<?, ? examples/s]

Map:   0%|          | 0/1523 [00:00<?, ? examples/s]

In [ ]:
roberta_model = AutoModelForSequenceClassification.from_pretrained(
# 載入預訓練好的RoBERTa-base模型，準備拿來做二元分類

    roberta_name,
    # 指定模型名稱，這裡用的是roberta-base

    num_labels=2
    # 設定輸出類別數為2，對應災難與非災難
)

rb_args = TrainingArguments(
# 設定 RoBERTa 訓練時會用到的所有參數

    output_dir="./roberta_out",
    # 設定模型訓練過程與結果輸出的資料夾位置

    learning_rate=1e-5,
    # 設定微調transformer常用的學習率，避免模型權重被破壞

    per_device_train_batch_size=16,
    # RoBERTa比較吃記憶體，所以訓練時batch size設小一點

    per_device_eval_batch_size=32,
    # 驗證時batch size設大一點，加快評估速度

    num_train_epochs=3,
    # 設定訓練跑2個epoch，避免過擬合也節省時間

    weight_decay=0.01,

    seed=SEED,
    # 固定亂數種子，確保結果可以重現

    report_to="none"
    # 關閉wandb等外部紀錄工具，避免在Colab出錯
)

rb_trainer = Trainer(
# 建立HuggingFace Trainer，負責整個RoBERTa的訓練流程

    model=roberta_model,
    # 指定要訓練的模型是剛剛載入的RoBERTa

    args=rb_args,
    # 傳入上面設定好的訓練參數

    train_dataset=ds_train_rb,
    # 指定訓練資料集

    eval_dataset=ds_val_rb,
    # 指定validation資料集，用來評估模型表現

    tokenizer=roberta_tokenizer,
    # 指定對應的tokenizer，讓Trainer正確處理輸入文字

    compute_metrics=compute_metrics
    # 使用前面定義好的F1-score作為模型評估指標
)

rb_trainer.train()
# 正式開始對RoBERTa進行fine-tuning訓練

eval_rb = rb_trainer.evaluate()
# 在validation資料集上評估訓練完成後的模型表現

f1_rb = eval_rb["eval_f1"]
# 從評估結果中取出RoBERTa的validation F1分數

print("RoBERTa | F1:", round(f1_rb, 4))
# 印出RoBERTa的F1分數，用來和前面模型做比較

# 用HuggingFace Trainer微調RoBERTa-base，訓練設定和DistilBERT類似，只是因為模型比較大，所以batch size調小，最後用validation F1來比較模型表現

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2443011623.py:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  rb_trainer = Trainer(


Step,Training Loss
500,0.442800
1000,0.332900


RoBERTa | F1: 0.822


In [ ]:
pd.DataFrame({
# 建立一個pandas DataFrame，用來整理不同模型的驗證結果，方便比較

    "Model": [
    # 模型名稱，照實驗順序列出來

        "TF-IDF + LR",
        # 傳統文字baseline：TF-IDF搭配Logistic Regression

        "CountVec + NB",
        # 傳統文字baseline：CountVectorizer搭配Naive Bayes

        "DistilBERT",
        # 第一個深度學習模型，使用較輕量的DistilBERT

        "RoBERTa"
        # 延伸實驗模型，使用較大的RoBERTa-base
    ],

    "Validation F1": [
    # 每個模型在validation set上的F1-score

        f1_lr,
        # TF-IDF+Logistic Regression的validation F1分數

        f1_nb,
        # CountVectorizer + Naive Bayes的validation F1分數

        f1_distil,
        # DistilBERT fine-tuning後的validation F1分數

        f1_rb
        # RoBERTa fine-tuning後的validation F1分數
    ]
})

# 把所有模型在validation上的F1分數整理成表格，方便直接比較傳統方法跟transformer模型的效果差異，也用來決定最後要送Kaggle的模型

,Model,Validation F1
0,TF-IDF + LR,0.768233
1,CountVec + NB,0.759124
2,DistilBERT,0.806791
3,RoBERTa,0.822047


In [ ]:
def roberta_predict(texts, threshold=0.65):
# 定義一個用RoBERTa做預測的函式，threshold用來控制判成災難的門檻

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # 檢查目前環境有沒有GPU，有就用GPU，加快推論速度，沒有就用CPU

    roberta_model.to(device)
    # 把RoBERTa模型移到同一個device，避免model跟資料在不同地方出錯

    enc = roberta_tokenizer(
        list(texts),
        # 把輸入的文字轉成list，確保tokenizer可以正常一次處理多筆

        padding=True,
        # 自動補齊句子長度，讓batch內每一句長度一致

        truncation=True,
        # 如果句子超過最大長度，就直接截斷，避免超出模型限制

        max_length=128,
        # 每一句最多只保留128個token，和訓練時設定一致

        return_tensors="pt"
        # 指定輸出為PyTorch tensor，才能直接丟進模型
    )

    enc = {k: v.to(device) for k, v in enc.items()}
    # 把tokenizer產生的所有tensor一起搬到同一個device（GPU 或 CPU）

    roberta_model.eval()
    # 把模型切換成evaluation模式，關閉dropout，確保推論結果穩定

    with torch.no_grad():
    # 關閉梯度計算，因為現在只是做預測，不需要反向傳播

        logits = roberta_model(**enc).logits
        # 把編碼後的文字丟進 RoBERTa，取得模型輸出的 logits（尚未轉成機率）

        probs = torch.softmax(logits, dim=1)[:, 1]
        # 用softmax把logits轉成機率，並只取「災難類別（label=1）」的機率

    preds = (probs >= threshold).long()
    # 只要災難機率 >= threshold，就判成1（災難），否則判成0（非災難）

    return preds.cpu().numpy(), probs.cpu().numpy()
    # 回傳最終的分類結果，以及對應的災難機率，並轉成numpy方便後續使用

In [ ]:
roberta_preds, roberta_probs = roberta_predict(test["input_rb"], threshold=0.65)
# 使用剛剛定義好的roberta_predict函式，對測試集文字做預測
# roberta_preds是0/1分類結果，roberta_probs是預測為災難的機率

submission = pd.DataFrame({
    "id": test["id"],
    # 建立submission的id欄位，必須和Kaggle提供的test.csv對齊

    "target": roberta_preds
    # 將RoBERTa預測出的分類結果放到target欄位
    # 1表示災難推文，0表示非災難推文
})

submission.to_csv("submission_roberta.csv", index=False)
# 將預測結果存成CSV檔案，格式符合Kaggle上傳規定

print("已輸出 submission_roberta.csv")
# 確認檔案已成功產生，可以直接拿去Kaggle上傳

# 這一步是用validation調好的threshold，讓RoBERTa在測試集上輸出最終預測結果，
# 再整理成Kaggle官方指定格式，作為leaderboard評分用的 submission

已輸出 submission_roberta.csv


In [ ]:
import gradio as gr

def roberta_predict_demo(text):
    # 把使用者輸入的文字包成list，因為tokenizer預期是批次輸入
    texts = [text]

    # 判斷目前環境是否有GPU，沒有就用CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 確保模型在同一個device上
    roberta_model.to(device)
    roberta_model.eval()

    # 將文字轉成RoBERTa可以吃的token格式
    enc = roberta_tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    # 把所有輸入tensor搬到同一個device
    enc = {k: v.to(device) for k, v in enc.items()}

    # 不計算梯度，單純做推論
    with torch.no_grad():
        logits = roberta_model(**enc).logits
        probs = torch.softmax(logits, dim=1)

    # 取預測為「災難」的機率
    prob_disaster = float(probs[0, 1])

    # 用0.5當分類門檻
    label = "Disaster" if prob_disaster >= 0.5 else "Not a disaster "

    return {
        "prediction": label,
        "confidence": round(prob_disaster, 3)
    }

with gr.Blocks() as demo:
    gr.Markdown("# 災難推文分類平台（RoBERTa）")
    gr.Markdown("輸入一段推文，平台會即時判斷是否為災難相關內容")

    inp = gr.Textbox(label="輸入推文文字")
    btn = gr.Button("預測")
    out = gr.JSON(label="模型輸出")

    btn.click(
        fn=roberta_predict_demo,
        inputs=inp,
        outputs=out
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://02813df2db472b1b0f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
